In [3]:
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import pickle
import warnings
warnings.filterwarnings("ignore")
import joblib
import numpy as np
import pandas as pd

In [4]:
result_compare = pd.DataFrame([['Logistic Regression',"Bag of word(Unigram)+sentiment_score","-","-" ,0.691,],
                               ['LightGBM Classifier',"Bag of word(Unigram)+sentiment_score","-","-" ,0.720],
                               ['RandomForest Classifier',"Bag of word(Unigram)+sentiment_score","-","-" ,0.69],
                               ['XGBoost Classifier',"Bag of word(Unigram)+sentiment_score","0.659","-" ,0.657],
                               ['Logistic Regression',"Bag of word(Bigram)+sentiment_score","-","-" ,0.695,],
                               ['LightGBM Classifier',"Bag of word(Bigram)+sentiment_score","-","-" ,0.694],
                               ['RandomForest Classifier',"Bag of word(Bigram)+sentiment_score","-","-" ,0.619],
                               ['XGBoost Classifier',"Bag of word(Bigram)+sentiment_score","0.602","-" ,0.599],
                               ['Logistic Regression',"TF-IDF(Uni+Bigram)+sentiment_score","-","-" ,0.695,],
                               ['LightGBM Classifier',"TF-IDF(Uni+Bigram)+sentiment_score","-","-" ,0.694],
                               ['RandomForest Classifier',"TF-IDF(Uni+Bigram)+sentiment_score","-","-" ,0.626],
                               ['LSTM+Dense',"Text_sequnece_padded",0.705,"-" ,0.710],
                               ['bert_pretained_model',"Bert Tokeinizer",0.652,"-" ,0.660],
                               ['TFDistilBertForSequenceClassification',"DistilBertTokenizer(for cleaned text)",0.773,0.821 ,0.757],
                               ['TFDistilBertForSequenceClassification',"DistilBertTokenizer(without cleaned text)",0.785,0.827 ,0.771]],
                              columns = ['Model_Name','Text Features Type',"Train Acccurcay","CV_accuracy","Test_accuracy" ], index = list(range(1,16)))

display(result_compare)

,Model_Name,Text Features Type,Train Acccurcay,CV_accuracy,Test_accuracy
1,Logistic Regression,Bag of word(Unigram)+sentiment_score,-,-,0.691
2,LightGBM Classifier,Bag of word(Unigram)+sentiment_score,-,-,0.720
3,RandomForest Classifier,Bag of word(Unigram)+sentiment_score,-,-,0.690
4,XGBoost Classifier,Bag of word(Unigram)+sentiment_score,0.659,-,0.657
5,Logistic Regression,Bag of word(Bigram)+sentiment_score,-,-,0.695
6,LightGBM Classifier,Bag of word(Bigram)+sentiment_score,-,-,0.694
7,RandomForest Classifier,Bag of word(Bigram)+sentiment_score,-,-,0.619
8,XGBoost Classifier,Bag of word(Bigram)+sentiment_score,0.602,-,0.599
9,Logistic Regression,TF-IDF(Uni+Bigram)+sentiment_score,-,-,0.695
10,LightGBM Classifier,TF-IDF(Uni+Bigram)+sentiment_score,-,-,0.694


In [48]:
model = TFDistilBertForSequenceClassification.from_pretrained('./model/clf_distilBert_no_clean')
model_name, max_len = pickle.load(open('./model/info.pkl', 'rb'))

Some layers from the model checkpoint at ./model/clf_distilBert_no_clean were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./model/clf_distilBert_no_clean and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# tkzr = DistilBertTokenizer.from_pretrained(model_name)
# joblib.dump(tkzr,"./model/clf_distilBert_no_clean/tkzr.pkl")

In [51]:
# tflite_model_name   = "tflite_distilbert_mdoel.tflite"
# open(tflite_model_name, "wb").write(tflite_distilbert_mdoel)

In [52]:
# model = tf.lite.Interpreter(model_path = tflite_model_name)
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# print("Input Shape:", input_details[0]['shape'])
# print("Input Type:", input_details[0]['dtype'])
# print("Output Shape:", output_details[0]['shape'])
# print("Output Type:", output_details[0]['dtype'])

In [53]:
# #interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
# #interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
# model.allocate_tensors()
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# print("Input Shape:", input_details[0]['shape'])
# print("Input Type:", input_details[0]['dtype'])
# print("Output Shape:", output_details[0]['shape'])
# print("Output Type:", output_details[0]['dtype'])

In [54]:
tkzr= joblib.load("./model/clf_distilBert_no_clean/tkzr.pkl")

In [55]:
def construct_encodings(x, tkzr, max_len, trucation=True, padding=True):
    return tkzr(x, max_length=max_len, truncation=trucation, padding=padding)

In [56]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

In [57]:
def create_predictor(model,tkzr, max_len):
    
    #tkzr = DistilBertTokenizer.from_pretrained(model_name)
    tkzr= tkzr
    
    def predict_proba(text):
        
        x = [text]

        encodings = construct_encodings(x, tkzr, max_len=max_len)
        tfdataset = construct_tfdataset(encodings)
        tfdataset = tfdataset.batch(1)

        preds = model.predict(tfdataset).logits
        preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
        return preds[0]

    return predict_proba



#prediction=clf("You mean he doesnt get to retroactively remove consent like women with buyers remorse are allowed to?")

In [58]:
# try:
Sample_text= str(input("Enter a text to check if it is Sarcastic : \n \n "))
print("\n") 
clf = create_predictor(model, tkzr, max_len)
prediction=clf(Sample_text)
print(f"Probablity of this text to be non sarcastic: {prediction[0]}")
print("\n")
print(f"Probablity of this text to be sarcastic    : {prediction[1]}")
print("\n")
label_= np.argmax(np.array(prediction))
#out=""
if label_==1:
    print("We have detected Sarcasm in this comment")
else:
    print("We couldn't detected 'Sarcasm' in this comment")
    
# except:
#     print("Please enter a string ")

Enter a text to check if it is Sarcastic : 
 
 You mean he doesnt get to retroactively remove consent like women with buyers remorse are allowed to?


Probablity of this text to be non sarcastic: 0.12029655277729034


Probablity of this text to be sarcastic    : 0.8797035217285156


We have detected Sarcasm in this comment
